In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'torch'

In [ ]:
def target_function(x, with_error = False):
    y = np.sin(x) 
    if with_error:
        return y + np.random.random(x.shape) * 0.1 - 0.05
    else:
        return y
    

In [ ]:
x_min, x_max = -2 * np.pi, 2 * np.pi
num_points = 100
x_data = np.linspace(x_min, x_max, num_points)
y_data = target_function(x_data, True)

In [ ]:
input_size = 1
hidden_size = 30
output_size = 1

In [ ]:
# Weight initialization
torch.menual_seed(41)
W1 = torch.randn(input_size, hidden_size, requires_grad=False) * 0.5
b1 = torch.zeros(1, hidden_size, requires_grad=False)
W2 = torch.randn(hidden_size, output_size, requires_grad=False) * 0.5
b2 = torch.zeros(1, output_size, requires_grad=False)

X = torch.tensor(x_data, dtype=torch.float32)
Y = torch.tensor(y_data, dtype=torch.float32)

NameError: name 'torch' is not defined

In [ ]:
# Activation function
def activation(x):
    x = torch.clamp(x, -500, 5000)
    return 1 / (1 + torch.exp(-x))

# Derivative of the activation function
def activation_derivative(x):
    return x * (1 - x)

In [ ]:
# Linear distribution/Expansion
def predict(X, W1, b1, W2, b2):
    z1 = torch.mm(X, W1) + b1
    a1 = activation(z1)
    z2 = torch.mm(X, W2) + b2
    y_pred = z2
    return z1, a1, z2, y_pred

In [ ]:
# Inverse distribution error
def backward(X, Y, z1, a1, z2, y_pred, W2):
    m = X.size(0)
    dZ2 = y_pred - Y
    dW2 = torch.mm(dZ2, a1)
    db2 = torch.sum(dZ2, dim=0, keepdim=True) / m

    dA1 = torch.mm(dZ2, W2.t())
    dZ1 = dA1 * activation_derivative(a1)

    dW1 = torch.mm(X.t(), dZ1) / m
    db1 = torch.sum(dZ1, dim=0, keepdim=True) / m

    return dW1, db1, dW2, db2


In [ ]:
# Update weights
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [ ]:
# Hyperparameters of the NN
learning_rate = 0.25
epoches = 1000

In [ ]:
# Loss function
def compute_loss(y_pred, Y):
    return torch.mean((y_pred - Y) ** 2)

In [ ]:
# Training the NN
def train(X, Y, W1, b1, W2, b2, learning_rate, epochs):
    losses = []
    for epoch in range(epochs):
        # linear distribution
        z1, a1, z2, y_pred = predict(X, W1, b1, W2, b2)

        # Compute loss function
        loss = compute_loss(y_pred, Y)
        losses.append(loss.item())

        # inverse linear distribution
        dW1, db1, dW2, db2 = backward(X, Y, z1, a1, z2, y_pred, W2)

        # Update parameters
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        # Output information on leaarning progress
        if epoch % 200 == 0:
            print(f"Epoch {epoch}: loss {loss.item():.6f}")

        return W1, b1, W2, b2, losses


In [ ]:
# Learning
W1, b1, W2, b2, losses = train(X, Y, W1, b1, W2, b2, learning_rate, epoches)

with torch.no_grad():
    _, _, _, y_pred = predict(X, W1, b1, W2, b2)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(x_data, y_data, label="True function", color="blue", marker="o")
plt.plot(x_data, y_pred.numpy(), label="Approximate", color="red", linestyle="--")
plt.legend()
plt.grid(True)
plt.title("Approximation function")
plt.xlabel("x")
plt.ylabel("y")

plt.subplot(1, 2, 2)
plt.plot(losses, color="green") # losses[5000:]
plt.title("Loss function")
plt.xlabel("Num of epochs")
plt.ylabel("MSE")
plt.grid(True)

plt.tight_layout()
plt.show()

final_loss = compute_loss(y_pred, Y)
print(f"\nFinal loss (MSE): {final_loss.item():.6f}")